In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
weight = 0.7
bias = 0.3
X = torch.arange(0,1,0.02).unsqueeze(dim = 1)
y = weight*X + bias

In [ ]:
X.shape, y.shape

In [ ]:
train_split = int(0.8* X.shape[0])
X_train, y_train, X_test, y_test = X[:train_split], y[:train_split], X[train_split:], y[train_split:]
train_split

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
def plot_data(train_data = X_train,
              train_labels = y_train,
              test_data = X_test,
              test_labels = y_test,
              predictions = None):
  plt.figure(figsize = (9,4))
  plt.scatter(train_data,train_labels, c='b',s=4,label="Training Data")
  plt.scatter(test_data,test_labels, c='g',s=4,label="Test Data")
  if predictions is not None:
    plt.scatter(test_data,predictions, c='r',s=4,label="Predictions")
  plt.legend()


In [ ]:
plot_data();

## Pytorch model building essentials:
* torch.nn :Contains all the building blocks for computational graphs (a neural network can be considered a computational graph)
* nn.Parameter: what parameters our model try and learn often pytorch layer from torch.nn will set this for us
* torch.nn.Module - The base class for all NN modules, if you subclass it you should override forward() method
* torch.optim - This is where the optimizers in Pytorch live, they will help with gradient descent
* def forward() : All nn.Module subclasses must override  forward(), this method defines what happens in the forward computation

In [ ]:
#Building the model:
class LinearRegressionModel(nn.Module): #nn.Module
  def __init__(self):
    super().__init__()
    self.weights = nn.Parameter(torch.randn(1,
                                            requires_grad = True,
                                            dtype = torch.float))
    self.bias = nn.Parameter(torch.randn(1,
                                         requires_grad = True,

                                         dtype = torch.float))
  def forward(self,X:torch.Tensor)->torch.Tensor:
    forward_pass = self.weights * X + self.bias
    return forward_pass


In [ ]:
#create a random seed first because the model uses random values for our weights and biases.
torch.manual_seed(42)
model_0 = LinearRegressionModel()


In [ ]:
print(list(model_0.parameters()))
print(model_0.state_dict())

In [ ]:
X_test,y_test

In [ ]:
## making predictions with torch.inference_mode()
#we could also do it without using the inference_mode() and just running y_preds = model_0(X_test) but if we use inference_mode() it will not calculate gradients. You can also use torch.no_grad()
with torch.inference_mode():
  y_preds = model_0(X_test)

# with torch.no_grad():
#   y_preds = model_0(X_test)

y_test,y_preds

In [ ]:
# y_preds = model_0(X_test)
# y_preds

In [ ]:
plot_data(predictions=y_preds)

In [ ]:
# Train the model: The whole idea of training is the model to move from *unknown* parameters to  known representation
# A loss function is a function to measure how wrong your predictions are from the standard values
# what we need to train:
#  - A loss function
# - Optimizer : Takes into action the loss of a model and adjusts the model's parameters (weights and biases)
# We need a training loop and a test loop
list(model_0.parameters())
model_0.state_dict()

In [ ]:
## setting up a loss function
loss_fn = nn.L1Loss()
# set up an optimizer
optimizer = torch.optim.SGD(params= model_0.parameters(),lr=0.01)

In [ ]:
#building a training loop and testing loop:
'''
0. Loop through the data
1. Forward passs (this involves data moving through models forward() method) to make predictions
2. Calculate the loss (compare forward pass predictions to groud truth labels)
3. Optimizer zero grad
4. Loss backward- move backwards through the network to calculate the gradients of each of the parameters with respect to the loss (backpropagation)
5. Optimizer step: Use the optimizer to adjust our model's parameters to improve the loss (gradient descent)
'''

In [ ]:
#training
#epoch is one loop through data
torch.manual_seed(42)
epochs = 150 # this is also a hyperparameter

#track values
epoch_count = []
loss_values = []
test_loss_values = []
# 0. loop through the data
for epoch in range(epochs):
  model_0.train() #train mode in pytorch sets all parameters that requires gradients to requrire gradients
  #forward pass:
  y_pred = model_0(X_train)
  #calculate loss:
  loss = loss_fn(y_pred, y_train)
  loss_values.append(loss)
  # print(f"Training Loss:{loss}")
  #.Optimizer zero grad:
  optimizer.zero_grad()
  # Perform backpropagation with respect to the parameters of the model
  loss.backward()

  # step the optimizer (perform gradient descent)
  optimizer.step()

  model_0.eval() #turns of diff settins not needed for evaluation or testing
  with torch.inference_mode(): #turns off gradient tracking
    test_pred = model_0(X_test)

    #calculate the test loss:
    test_loss = loss_fn(test_pred,y_test)
    test_loss_values.append(test_loss)
    # print(f"Test loss:{test_loss}")

  epoch_count.append(epoch)
  if epoch % 10 ==0:

    print(f"Epoch:{epoch}\tTrain Loss:{loss}\tTest Loss: {test_loss}")
    #print(model_0.state_dict())

#print out model state_dict:
# print(model_0.state_dict())



In [ ]:
model_0.state_dict()

In [ ]:
#plot the loss curvers np.array(torch.tensor(loss_values).cpu().numpy())
plt.plot(np.array(torch.tensor(epoch_count).cpu().numpy()), np.array(torch.tensor(loss_values).cpu().numpy()), label = "Train Loss")
plt.plot(np.array(torch.tensor(epoch_count).cpu().numpy()), np.array(torch.tensor(test_loss_values).cpu().numpy()), label = "Test Loss")
plt.title("Training and Test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
with torch.inference_mode():
  y_preds_new = model_0(X_test)
plot_data(predictions=y_preds_new)

In [ ]:
#saving our model:
'''
There are three main methods to saving and loading our model:
1. torch.save() - allows you to save a Pythons pickle format
2.torch.load() - load your saved Pytorch Object
3. torch.nn.Module.load_state_dict() - load model's saved state dictionary
'''

In [ ]:
for param in model_0.state_dict():
  print(param,model_0.state_dict()[param].shape)



In [ ]:
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
torch.save(model_0.state_dict(), '/LR_model.pt')

In [ ]:
SAVE_PATH = "models/LR_model_ckpoint.pt"
torch.save({
            'epoch': epoch,
            'model_state_dict': model_0.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, SAVE_PATH)

In [ ]:
loaded_model = LinearRegressionModel()
loaded_model.load_state_dict(torch.load("/LR_model.pt"))
loaded_model.eval()

In [ ]:
checkpoint = torch.load(SAVE_PATH)
print(checkpoint['loss'])
print(checkpoint['epoch'])
print(checkpoint['model_state_dict'])
print(checkpoint['optimizer_state_dict'])


In [ ]:
loaded_model.state_dict()

In [ ]:
# make predictions wich loaded_model
loaded_model.eval()
with torch.inference_mode():
  loaded_preds = loaded_model(X_test)


In [ ]:
y_preds

In [ ]:
loaded_preds

In [ ]:
model_0.eval()
with torch.inference_mode():
  test_preds = model_0(X_test)
test_preds

In [ ]:
# Wrapping everyting up

In [ ]:
import torch
from torch import nn
import numpy
import matplotlib.pyplot as plt

torch.__version__

In [ ]:
#write device agnostic code:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
#create data
weight = 0.7
bias = 0.3
X = torch.arange(0,1,0.002).unsqueeze(dim=1)
y = weight * X + bias
X.shape,y.shape

In [ ]:
#spplit the data
train_split = int(0.8*X.shape[0])
X_train, y_train, X_test, y_test = X[:train_split], y[:train_split], X[train_split:], y[train_split:]
X_train.shape ,y_train.shape, X_test.shape, y_test.shape

In [ ]:
def plot_data(train_data = X_train,
              train_labels = y_train,
              test_data = X_test,
              test_labels = y_test,
              predictions = None):
  plt.figure(figsize = (9,4))
  plt.scatter(train_data,train_labels, c='b',s=4,label="Training Data")
  plt.scatter(test_data,test_labels, c='g',s=4,label="Test Data")
  if predictions is not None:
    plt.scatter(test_data,predictions, c='r',s=4,label="Predictions")
  plt.legend()

plot_data()

In [ ]:
class LRModel(nn.Module):
  def __init__(self):
    super(LRModel,self).__init__()
    self.linear_layer = nn.Linear(in_features=1, out_features=1)

  def forward(self, x:torch.Tensor)->torch.Tensor:
    return self.linear_layer(x)


torch.manual_seed(42)

model_1 = LRModel()
model_1, model_1.state_dict()
model_1.to(device)

In [ ]:
next(model_1.parameters()).device

In [ ]:
#loss fn and optimizer":
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params = model_1.parameters(),lr = 0.001)


In [ ]:
#Training
epoch_list = []
losses = []
test_losses = []

torch.manual_seed(42)
model_1.train()
epochs = 800

#put data on the same device:
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

for epoch in range(epochs):
  model_1.train()
  epoch_list.append(epoch)
  y_pred = model_1(X_train)

  loss = loss_fn(y_pred, y_train)
  losses.append(loss)
  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  with torch.inference_mode():
    test_pred = model_1(X_test)
    test_loss = loss_fn(test_pred, y_test)
    test_losses.append(test_loss)

  if epoch % 10 == 0:
    print(f"Epoch:{epoch} | Train loss:{loss} | Test loss: {test_loss}")



In [ ]:
test_pred.shape

In [ ]:
X_test.shape

In [ ]:
#plot the loss curvers np.array(torch.tensor(loss_values).cpu().numpy())
plt.plot(np.array(torch.tensor(epoch_list).cpu().numpy()), np.array(torch.tensor(losses).cpu().numpy()), label = "Train Loss")
plt.plot(np.array(torch.tensor(epoch_list).cpu().numpy()), np.array(torch.tensor(test_losses).cpu().numpy()), label = "Test Loss")
plt.title("Training and Test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
model_1.eval()
with torch.inference_mode():
  y_preds_1 = model_1(X_test)


In [ ]:
#checkout model preds
plot_data(predictions=y_preds_1.cpu())

In [ ]:
from pathlib import Path
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents= True, exist_ok=True)
MODEL_NAME = 'LR_model_01.pth'
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
MODEL_SAVE_PATH

print(f"Saving the model to {MODEL_SAVE_PATH}")
torch.save(obj = model_1.state_dict(), f=MODEL_SAVE_PATH)

In [ ]:
# Load the model:

loaded_model_1 = LRModel()
loaded_model_1.load_state_dict(torch.load(MODEL_SAVE_PATH))
loaded_model_1.to(device)
loaded_model_1.state_dict()

In [ ]:
loaded_model_1.eval()
with torch.inference_mode():
  y_preds_loaded = loaded_model_1(X_test)
y_preds_loaded == y_preds_1

In [ ]:
#exercises: